In [2]:
# importing packages and modules
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import minimize
from utils import *
from plotting_utils import *
from analysis_utils import *
from io_utils import *
import dynamic_glmhmm
from scipy.stats import multivariate_normal, norm

sns.set_context("talk")

ibl_data_path = '../data_IBL'
colorsFeatures = [['#FAA61A','indigo','#99CC66','#59C3C3','#9593D9'],['#FAA61A',"#2369BD","#A9373B",'#99CC66','#59C3C3','#9593D9']]
colorsStates = ['darkorange','darkblue','forestgreen','purple']
myFeatures = [['bias','delta stimulus', 'previous choice', 'previous reward'],['bias','contrast left','contrast right', 'previous choice', 'previous reward']]
sns.set_context("talk")
colormap = ['rebeccapurple','teal','darkturquoise','springgreen']

In [3]:
# NO LONGER WORKING, FIND NEW WAY TO DONWLAOD DATA


# # Downloading data, then unzipping manually in folder, then preprocessing with below code, adapted from Psytrack notebook (Roy et al., 2021)

# current_cwd = os.getcwd()
# SPATH = current_cwd 
# ibl_data_path = SPATH + '/../data_IBL'
# os.chdir(ibl_data_path)
# print(os.getcwd())

# !pip install ibllib
# # !wget -nc -O "{SPATH}ibl-behavior-data-Dec2019.zip" "https://ndownloader.figshare.com/files/21623715"
# # !unzip -d "{SPATH}" -n "{SPATH}ibl-behavior-data-Dec2019.zip"

# from ibllib import ONE


# import subprocess

# def runcmd(cmd, verbose = False, *args, **kwargs):

#     process = subprocess.Popen(
#         cmd,
#         stdout = subprocess.PIPE,
#         stderr = subprocess.PIPE,
#         text = True,
#         shell = True
#     )
#     std_out, std_err = process.communicate()
#     if verbose:
#         print(std_out.strip(), std_err)
#     pass

# runcmd('echo "Hello, World!"', verbose = True)

# runcmd("wget https://ndownloader.figshare.com/files/21623715", verbose = True)

# # Search all sessions that have these dataset types.
# required_vars = ['_ibl_trials.choice', '_ibl_trials.contrastLeft',
#                  '_ibl_trials.contrastRight','_ibl_trials.feedbackType']
# one = ONE()
# eids = one.search(required_vars)

# mouseData = pd.DataFrame()
# for eid in eids:
#     lab, _, subject, date, session = eid.split("/")    
#     sess_vars = {
#         "eid": eid,
#         "lab": lab,
#         "subject": subject,
#         "date": date,
#         "session": session,
#     }
#     mouseData = mouseData.append(sess_vars, sort=True, ignore_index=True)

# all_vars = ["contrastLeft", "contrastRight", "choice", "feedbackType", "probabilityLeft"]
# df = pd.DataFrame()

# all_mice = []
# for j, s in enumerate(mouseData["subject"].unique()):
#     print("\rProcessing " + str(j+1) + " of " + str(len(mouseData["subject"].unique())), end="")
#     mouse = mouseData[mouseData["subject"]==s].sort_values(['date', 'session']).reset_index()
#     for i, row in mouse.iterrows():
#         myVars = {}
#         for v in all_vars:
#             filename = "_ibl_trials." + v + ".npy"
#             var_file = os.path.join(ibl_data_path, row.eid, "alf", filename)
#             myVars[v] = list(np.load(var_file).flatten())

#         num_trials = len(myVars[v])
#         myVars['lab'] = [row.lab]*num_trials
#         myVars['subject'] = [row.subject]*num_trials
#         myVars['date'] = [row.date]*num_trials
#         myVars['session'] = [row.session]*num_trials

#         all_mice += [pd.DataFrame(myVars, columns=myVars.keys())]
        
# df = pd.concat(all_mice, ignore_index=True)

# df = df[df['choice'] != 0]        # dump mistrials
# df = df[df['feedbackType'] != 0]  # 3 anomalous trials from ZM_1084, omit
# df.loc[np.isnan(df['contrastLeft']), "contrastLeft"] = 0
# df.loc[np.isnan(df['contrastRight']), "contrastRight"] = 0
# df.loc[df["contrastRight"] < 0, "contrastLeft"] = np.abs(df.loc[df["contrastRight"] < 0, "contrastRight"])
# df.loc[df["contrastRight"] < 0, "contrastRight"] = 0  # 81 anomalous trials in CSHL_002, correct
# df["correctSide"] = df["feedbackType"] * df["choice"]      # new column to indicate correct answer
# df.loc[df["correctSide"]==1, "correctSide"] = 0
# df.loc[df["correctSide"]==-1, "correctSide"] = 1
# df.loc[df["feedbackType"]==-1, "feedbackType"] = 0
# df.loc[df["choice"]==1, "choice"] = 0
# df.loc[df["choice"]==-1, "choice"] = 1
# df.to_csv(SPATH+"ibl_processed.csv", index=False)

In [4]:
dfAll = pd.read_csv(ibl_data_path + '/Ibl_processed.csv')
dfAll.head()

,contrastLeft,contrastRight,choice,feedbackType,probabilityLeft,lab,subject,date,session,correctSide
0,1.0,0.0,1,0,1.0,angelakilab,IBL-T1,2019-02-09,1,0
1,1.0,0.0,1,0,1.0,angelakilab,IBL-T1,2019-02-09,1,0
2,0.0,1.0,1,1,1.0,angelakilab,IBL-T1,2019-02-09,1,1
3,0.5,0.0,0,1,0.5,angelakilab,IBL-T1,2019-02-09,1,0
4,0.5,0.0,0,1,0.9,angelakilab,IBL-T1,2019-02-09,1,0


In [5]:
# only looking at animals from 3 labs (32 total animals)
labChosen =  ['angelakilab','churchlandlab','wittenlab']
subjectsAll = []
for lab in labChosen:
    subjects = np.unique(dfAll[dfAll['lab'] == lab]['subject']).tolist()
    subjectsAll = subjectsAll + subjects

# removing missing or incomplete animals
if ('NYU-01' in subjectsAll):
    subjectsAll.remove('NYU-01')
if ('NYU-06' in subjectsAll):
    subjectsAll.remove('NYU-06')
if ('CSHL_007' in subjectsAll):
    subjectsAll.remove('CSHL_007')
if ('CSHL049' in subjectsAll):
    subjectsAll.remove('CSHL049')

In [6]:
x, y, sessInd, correctSide = get_mouse_design(dfAll, subject='ibl_witten_15', sessStop=None, signedStimulus=False, pTanh=5)

In [26]:
# # concatanating post-training data across half the mice into one global dataset
# firstSubject = 'ibl_witten_02'
# signedStimulus = True
# x, y, sessInd, correctSide = get_mouse_design(dfAll, subject=firstSubject, sessStop=None, signedStimulus=signedStimulus, pTanh=5) 
# c = 0
# for subject in subjectsAll:
#     if (subject != firstSubject and c%2==0):
#         xTemp, yTemp, sessIndTemp, correctSideTemp = get_mouse_design(dfAll, subject=subject, sessStop=None, signedStimulus=signedStimulus, pTanh=5) 

#         # only using post-training data
#         _, _, _, firstBlockSession = get_design_biased_blocks(dfAll, subject, sessIndTemp, sessStop=None) # session of first biased block introduced & end of training
#         if firstBlockSession is not np.nan:
#             firstIndex = sessIndTemp[firstBlockSession]
#             xTemp = xTemp[firstIndex:]
#             yTemp = yTemp[firstIndex:]
#             sessIndTemp = sessIndTemp[firstBlockSession:]
#             sessIndTemp = [i - firstIndex for i in sessIndTemp]

#             x = np.concatenate((x,xTemp))
#             y = np.concatenate((y,yTemp))
#             sessInd = sessInd + [i + sessInd[-1] for i in sessIndTemp[1:]]
    
#     c += 1
# print(x.shape)
# print(y.shape)
# np.save(f'../data_IBL/X_allAnimals_signedStimulus={signedStimulus}', x)
# np.save(f'../data_IBL/Y_allAnimals_signedStimulus={signedStimulus}', y)
# np.save(f'../data_IBL/sessInd_allAnimals_signedStimulus={signedStimulus}', sessInd)

signedStimulus = False
x = np.load(f'../data_IBL/X_allAnimals_signedStimulus={signedStimulus}.npy')
y = np.load(f'../data_IBL/Y_allAnimals_signedStimulus={signedStimulus}.npy')
sessInd = np.load(f'../data_IBL/sessInd_allAnimals_signedStimulus={signedStimulus}.npy')

In [27]:
print(x.shape)
print(y.shape)
print(sessInd[-1])

(629730, 5)
(629730,)
629730


In [28]:
dGLMHMM = dynamic_glmhmm.dynamic_GLMHMM(x.shape[0],1,x.shape[1],2)
present = np.ones((x.shape[0])).astype(int) # using all data
P, pi, w = dGLMHMM.generate_param(sessInd=sessInd, transitionDistribution=['dirichlet', (5, 1)], weightDistribution=['uniform', (-2,2)], model_type='standard') 
phi = dGLMHMM.observation_probability(x, w)
print(P.shape)
print(y.shape)
alpha, ct, ll = dGLMHMM.forward_pass(y, present, P, pi, phi, sessInd[:-1])

(629730, 1, 1)
(629730,)
